# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv("Orders.zip")
orders.head()



,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here
# Sub Problem 1:
agg_custom = orders.groupby(["CustomerID"]).agg({"amount_spent":"sum"})
agg_custom.sort_values(by="amount_spent", ascending=False)


,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06
...,...
17956,12.75
16454,6.90
14792,6.20


In [8]:
# Sub Problem 2
quantile = pd.qcut(agg_custom["amount_spent"],[0.75,0.95,1], labels = ["PREFERRED", "VIP"])
quantile.value_counts()

PREFERRED    868
VIP          217
Name: amount_spent, dtype: int64

In [16]:
# Sub Problem 3
quantile = pd.qcut(agg_custom["amount_spent"],[0.75,0.95,1], labels = ["PREFERRED", "VIP"])
quantile



CustomerID
12346          VIP
12347    PREFERRED
12348    PREFERRED
12349    PREFERRED
12350          NaN
           ...    
18280          NaN
18281          NaN
18282          NaN
18283    PREFERRED
18287    PREFERRED
Name: amount_spent, Length: 4339, dtype: category
Categories (2, object): ['PREFERRED' < 'VIP']

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [18]:
# your code here
orders["Type of Client"] = quantile
orders

orders_vip = orders[orders["Type of Client"] == 'VIP']
orders_vip.groupby(["Country", "Type of Client"]).agg({"CustomerID":"count"}).sort_values(by="CustomerID", ascending=False)

,,CustomerID
Country,Type of Client,
United Kingdom,VIP,191
France,VIP,6
Germany,VIP,6
Norway,VIP,6
EIRE,VIP,3
Portugal,VIP,2
Spain,VIP,2
Japan,VIP,1
EIRE,PREFERRED,0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [20]:
# your code here
orders = orders.groupby(["Country"]).agg({"Type of Client":"count"})
orders.sort_values(by="Type of Client", ascending=False)


,Type of Client
Country,
United Kingdom,971
Norway,24
Germany,22
France,19
Spain,15
EIRE,12
Denmark,8
Japan,6
Portugal,6
